<a href="https://colab.research.google.com/github/dalsi137/nlpJ036/blob/master/NLP_tfidf_J036.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import classification_report, roc_auc_score
import string
from string import digits
from nltk.corpus import stopwords
import lightgbm as lgb

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [3]:
tweet = pd.read_csv('https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv', error_bad_lines=False)

twitter = tweet[tweet['Sentiment']!='irrelevant']

def clean(text):
  text= text.lower()
  text = text.translate(text.maketrans('','',digits))#removing digits
  text = text.translate(text.maketrans('', '', '!"@#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'))
  return text

twitter['TweetTokens']= twitter['TweetText'].apply(clean)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [0]:
vectorizer = CountVectorizer(lowercase=True, stop_words=set(stopwords.words('english')))
X = vectorizer.fit_transform(twitter['TweetTokens'])

In [0]:
features = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [0]:
X_train,X_test,y_train,y_test = tts(X,twitter['Sentiment'],test_size=0.3, random_state=30)

In [0]:
x_train = pd.DataFrame(X_train.toarray(),columns=vectorizer.get_feature_names())
x_test = pd.DataFrame(X_test.toarray(),columns=vectorizer.get_feature_names())

In [0]:
ran = RandomForestClassifier()

In [9]:
ran.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [10]:
y_pred = ran.predict(x_test)
print(classification_report(ran.predict(x_test),y_test))

              precision    recall  f1-score   support

    negative       0.43      0.71      0.54       105
     neutral       0.94      0.78      0.85       870
    positive       0.22      0.53      0.31        53

    accuracy                           0.76      1028
   macro avg       0.53      0.68      0.57      1028
weighted avg       0.85      0.76      0.79      1028



In [0]:
def multiclass_roc_auc_score(y_test, y_pred, average='macro'):
  lb = LabelBinarizer()
  lb.fit(y_test)
  Y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return roc_auc_score(Y_test, y_pred, average=average)

In [12]:
multiclass_roc_auc_score(y_test, y_pred)

0.6491925846554244

In [0]:
le = LabelEncoder()
Y_test = le.fit_transform(y_test)

In [14]:
pred_ran = ran.predict_proba(x_test)
pred_ran

array([[0.03      , 0.95788889, 0.01211111],
       [0.        , 0.26      , 0.74      ],
       [0.14      , 0.68      , 0.18      ],
       ...,
       [0.        , 0.84552146, 0.15447854],
       [0.29      , 0.70746753, 0.00253247],
       [0.        , 0.90951477, 0.09048523]])

In [0]:
params = {
    "objective" : "multiclass",
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':3,
    'learning_rate':0.01,
    'max_depth': 7,
    'num_leaves': 127,
    'feature_fraction': 0.4,
    'bagging_freq': 10,
    'num_iterations':1000 ,
    'max_bin' : 32}

In [0]:
le = LabelEncoder()  
d_train = lgb.Dataset(x_train,le.fit_transform(y_train))

In [17]:
clf = lgb.train( params,d_train, 100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [0]:
preds = clf.predict(x_test)
best_preds= [np.argmax(line) for line in preds]

In [20]:
np.bincount((le.fit_transform(y_train)))

array([ 397, 1605,  394])

In [21]:
print(classification_report(le.inverse_transform(best_preds),y_test))

              precision    recall  f1-score   support

    negative       0.25      0.56      0.34        77
     neutral       0.94      0.75      0.83       922
    positive       0.09      0.38      0.14        29

    accuracy                           0.72      1028
   macro avg       0.43      0.56      0.44      1028
weighted avg       0.87      0.72      0.78      1028



In [22]:
np.bincount((best_preds))

array([ 77, 922,  29])

In [23]:
preds = clf.predict(x_train)
best_preds_train= [np.argmax(line) for line in preds]
print(classification_report(le.inverse_transform(best_preds_train),y_train))

              precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396



In [24]:
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_1(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.6:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.6 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_1(preds)),y_train))

Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396


 0.6 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.12      0.85      0.21        55
     neutral       0.84      0.82      0.83      1637
    positive       0.59      0.33      0.43       704

    accuracy                           0.68      2396
   macro avg       0.52      0.67      0.49      2396
weighted avg       0.75      0.68      0.70      2396



In [25]:
pred_rf = ran.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(ran.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(le.inverse_transform(thresh_1(pred_rf)),y_test) )

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.43      0.71      0.54       105
     neutral       0.94      0.78      0.85       870
    positive       0.22      0.53      0.31        53

    accuracy                           0.76      1028
   macro avg       0.53      0.68      0.57      1028
weighted avg       0.85      0.76      0.79      1028


 0.6 threshold for rf : 
               precision    recall  f1-score   support

    negative       0.19      0.83      0.31        41
     neutral       0.85      0.84      0.85       733
    positive       0.54      0.26      0.35       254

    accuracy                           0.70      1028
   macro avg       0.53      0.65      0.50      1028
weighted avg       0.75      0.70      0.70      1028



In [26]:
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_2(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.7:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.7 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_2(preds)),y_train))

Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396


 0.7 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.07      0.90      0.12        29
     neutral       0.71      0.86      0.78      1327
    positive       0.75      0.28      0.41      1040

    accuracy                           0.61      2396
   macro avg       0.51      0.68      0.44      2396
weighted avg       0.72      0.61      0.61      2396



In [27]:
pred_rf = ran.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(ran.predict(x_test),y_test)))
print("\n 0.7 threshold for rf : \n",classification_report(le.inverse_transform(thresh_2(pred_rf)),y_test) )

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.43      0.71      0.54       105
     neutral       0.94      0.78      0.85       870
    positive       0.22      0.53      0.31        53

    accuracy                           0.76      1028
   macro avg       0.53      0.68      0.57      1028
weighted avg       0.85      0.76      0.79      1028


 0.7 threshold for rf : 
               precision    recall  f1-score   support

    negative       0.12      0.95      0.21        22
     neutral       0.76      0.86      0.81       640
    positive       0.64      0.22      0.33       366

    accuracy                           0.64      1028
   macro avg       0.51      0.68      0.45      1028
weighted avg       0.70      0.64      0.62      1028



In [28]:
print("Default Threshold for lgbm : \n",classification_report(le.inverse_transform(best_preds_train),y_train))
def thresh_3(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.8:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.8 Threshold for lgbm : \n",classification_report(le.inverse_transform(thresh_3(preds)),y_train))

Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396


 0.8 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.03      0.85      0.05        13
     neutral       0.42      0.92      0.58       737
    positive       0.92      0.22      0.35      1646

    accuracy                           0.44      2396
   macro avg       0.46      0.66      0.33      2396
weighted avg       0.76      0.44      0.42      2396



In [29]:
pred_rf = ran.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(ran.predict(x_test),y_test)))
print("\n 0.8 threshold for rf : \n",classification_report(le.inverse_transform(thresh_3(pred_rf)),y_test) )

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.43      0.71      0.54       105
     neutral       0.94      0.78      0.85       870
    positive       0.22      0.53      0.31        53

    accuracy                           0.76      1028
   macro avg       0.53      0.68      0.57      1028
weighted avg       0.85      0.76      0.79      1028


 0.8 threshold for rf : 
               precision    recall  f1-score   support

    negative       0.07      1.00      0.13        12
     neutral       0.61      0.90      0.73       493
    positive       0.81      0.19      0.31       523

    accuracy                           0.54      1028
   macro avg       0.50      0.70      0.39      1028
weighted avg       0.71      0.54      0.51      1028

